In [1]:
from pycrucible.pycrucible import *

In [2]:
apikey= SecureInput()

In [3]:
crux = CrucibleClient("https://crucible.lbl.gov/testapi", apikey)

In [ ]:
import requests

# Pull all the dataset IDs from the old Crucible API
old_api_url = "https://crucible.lbl.gov/api"
response = requests.get(f"{old_api_url}/datasets")
response.raise_for_status()

old_datasets = response.json()
old_dataset_ids = [dataset['id'] for dataset in old_datasets]
print(f"Found {len(old_dataset_ids)} datasets in old database")
print(f"First 10 dataset IDs: {old_dataset_ids[:10]}")

In [ ]:
# Check if each dataset exists in the new database
missing_datasets = []
existing_datasets = []

for dataset_id in old_dataset_ids:
    try:
        # Try to get the dataset from the new API
        dataset = crux.get_dataset(dataset_id)
        existing_datasets.append(dataset_id)
        print(f"✓ Dataset {dataset_id} exists in new database")
    except Exception as e:
        missing_datasets.append(dataset_id)
        print(f"✗ Dataset {dataset_id} missing from new database: {str(e)}")

print(f"\nSummary:")
print(f"Existing datasets: {len(existing_datasets)}")
print(f"Missing datasets: {len(missing_datasets)}")
print(f"Missing dataset IDs: {missing_datasets[:10]}...")  # Show first 10

In [ ]:
# if it does not exist -- add it
'''
pull JSON for dataset ID from cloud storage
upload new dataset
create any required access groups, projects, user entities, scientific metadata, thumbnails, associated files
'''

In [ ]:
# if it does exist -- check that the scientific metadata, associated files, owner, and project also exist

# if not add them